# Pandas - Select, groupby and merge with DataFrames

In [1]:
import os
import numpy as np
import pandas as pd

### Select

In [2]:
df = pd.DataFrame(np.random.rand(6,3), columns=list('abc'))
df[['a','c']]

,a,c
0,0.395989,0.013265
1,0.371867,0.404511
2,0.239524,0.254514
3,0.320249,0.352231
4,0.848869,0.363864
5,0.183527,0.935651


### Filter

In [3]:
idx = df.index[df['a']>0.5]
df.loc[idx,:]

,a,b,c
4,0.848869,0.453323,0.363864


### Mutate

In [4]:
idx = df.index[df['a']>0.5]
df.loc[idx,'b'] = 777
df

,a,b,c
0,0.395989,0.890191,0.013265
1,0.371867,0.455388,0.404511
2,0.239524,0.081585,0.254514
3,0.320249,0.311786,0.352231
4,0.848869,777.000000,0.363864
5,0.183527,0.381738,0.935651


### Group By

In [5]:
df = pd.DataFrame(np.random.rand(6,3), columns=list('abc'))
df['x'] = ['x1','x2','x1','x1','x3','x2']
# Gruppierung nach Spalte x (Spalte x als Index)
df.groupby('x').sum()

,a,b,c
x,,,
x1,2.237987,1.491367,1.701277
x2,0.721170,0.360798,0.952639
x3,0.630736,0.625138,0.654223


In [6]:
# Gruppierung nach Spalte x (Spalte x bleibt Spalte)
df.groupby('x', as_index=False).sum()

,x,a,b,c
0,x1,2.237987,1.491367,1.701277
1,x2,0.721170,0.360798,0.952639
2,x3,0.630736,0.625138,0.654223


In [7]:
# Gruppierung mit Fallunterscheidung
df.groupby(df['a']>0.5, as_index=False).sum()

,a,b,c
0,0.547245,0.951505,0.670759
1,3.042647,1.525798,2.637380


In [8]:
# mehrere Aggregationen
df.groupby('x', as_index=False).agg([np.sum, np.mean, np.size])

a                        b                        c               
         sum      mean size       sum      mean size       sum      mean size
x                                                                            
x1  2.237987  0.745996  3.0  1.491367  0.497122  3.0  1.701277  0.567092  3.0
x2  0.721170  0.360585  2.0  0.360798  0.180399  2.0  0.952639  0.476319  2.0
x3  0.630736  0.630736  1.0  0.625138  0.625138  1.0  0.654223  0.654223  1.0

In [9]:
# spezifische Ausgabe
dx = df.groupby('x', as_index=False).agg([np.sum, np.mean, np.size])
dx = dx.loc[:,[('a','sum'),('a','size')]]
dx.columns = ['sum','size']
dx

,sum,size
x,,
x1,2.237987,3.0
x2,0.721170,2.0
x3,0.630736,1.0


In [10]:
# mehrere Aggregationen von spezifischen Spalten
df.groupby('x', as_index=False).agg({'a':np.mean, 'b':np.sum, 'c':np.size})

,x,a,b,c
0,x1,0.745996,1.491367,3.0
1,x2,0.360585,0.360798,2.0
2,x3,0.630736,0.625138,1.0


### Merge / Join
https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html

In [11]:
# inner join
df1 = pd.DataFrame(np.random.rand(6,3), columns=list('abc'))
df1['x'] = ['x1','x2','x1','x1','x3','x2']
df2 = pd.DataFrame.from_dict({'x':['x1','x2','x3'], 'k2':[1,2,3], 'k3': [7,6,5]})
df1.merge(df2)
# oder
df1.merge(df2,on=['x'],how='inner')

,a,b,c,x,k2,k3
0,0.025189,0.469347,0.815153,x1,1,7
1,0.799727,0.928792,0.600180,x1,1,7
2,0.532540,0.161723,0.690560,x1,1,7
3,0.581948,0.470151,0.526415,x2,2,6
4,0.102395,0.364869,0.275039,x2,2,6
5,0.415562,0.178678,0.498008,x3,3,5


In [12]:
# inner join nach groupby
dx = df.groupby('x', as_index=False).sum()
df2.merge(dx)

,k2,k3,x,a,b,c
0,1,7,x1,2.237987,1.491367,1.701277
1,2,6,x2,0.721170,0.360798,0.952639
2,3,5,x3,0.630736,0.625138,0.654223


In [13]:
# inner join nach groupby (miitels Index)
dx = df.groupby('x').sum()
df2.merge(dx, left_on='x', right_index=True)

,k2,k3,x,a,b,c
0,1,7,x1,2.237987,1.491367,1.701277
1,2,6,x2,0.721170,0.360798,0.952639
2,3,5,x3,0.630736,0.625138,0.654223


In [14]:
# inner join mit unterschiedlichen Spalten Namen
df3 = pd.DataFrame.from_dict({'w':['x1','x2','x3'], 'k2':[1,2,3], 'k3': [7,6,5]})
df1.merge(df3,left_on=['x'],right_on=['w'],how='inner')

,a,b,c,x,k2,k3,w
0,0.025189,0.469347,0.815153,x1,1,7,x1
1,0.799727,0.928792,0.600180,x1,1,7,x1
2,0.532540,0.161723,0.690560,x1,1,7,x1
3,0.581948,0.470151,0.526415,x2,2,6,x2
4,0.102395,0.364869,0.275039,x2,2,6,x2
5,0.415562,0.178678,0.498008,x3,3,5,x3
